In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# Function to make bar graph from the data

def make_bar(data, ax):
    models = list(data.keys())  # List of every model tested
    models.sort()

    # Get a list of every test that was tested
    tests = set()
    for model in models:
        tests |= set(data[model].keys())

    tests = list(tests)
    tests.sort()

    tests = list(data[models[0]].keys())  # These will be the labels
    tests.sort()

    x = np.arange(len(tests))  # positions of labels

    # Make bar plots
    n = len(models)  # number of models tested
    width = 0.75 / n  # width of the bars
    for i in range(n):
        model = models[i]
        # Get the accuracy for each test
        accuracy = []
        sum_accuracy = 0  # Values used to calculate average accuracy
        count = 0
        for test in tests:
            if test in data[model].keys():  
                # This model was tested with this test
                accuracy.append(data[model][test])
                sum_accuracy += data[model][test]
                count += 1
            else:
                # This model didn't get tested with this test,
                # don't count it towards the average
                accuracy.append(0)

        # Calculate the average accuracy
        avg_accuracy = sum_accuracy / count
        label = '{:s} - average accuracy: {:.2f}'.format( model, avg_accuracy)
        rects = ax.bar(x + i*width, accuracy, width, label=label)


    ax.set_ylabel('Accuracy')
    ax.set_xlabel('Test')
    ax.set_xticks(x + (n-1)*width/2)
    ax.set_xticklabels(tests, rotation='vertical')
    ax.legend()
    ax.grid(True)

    return ax


    

In [3]:
# Function that loads data from the output json file and then calls make_bar

def load_data(directory):
    data = {}  # Initialize empty dictionary
    
    # Go through each file in the directory
    for filename in os.listdir(directory):
        # Make sure its a txt or json file
        if not filename.endswith('.txt') or filename.endswith('.json'):
            continue


        # Open the results json file
        f = open(directory + '/' + filename)
        f = json.load(f)

        # Store the information from the file in a dictionary
        for i in range(len(f)):
            accuracy = f[i]['Total Correct'] / f[i]['Total Attempted']

            model = f[i]['Model']
            # Remove the .txt from end of model name
            if model.endswith('.txt'):  
                model = model[:-4]

            # Remove the .txt from end of test name
            test = f[i]['Test']
            if test.endswith('.txt'):  
                test = test[:-4]
                
            # Create a new key if its not in the dictionary
            if model not in data.keys():  
                data[model] = {}
            
            # Add the accuracy for this test and model
            data[model].update({test: accuracy})  
    
    return data


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(40,10))

data = load_data('Experimental_Results')
ax = make_bar(data, ax)

plt.show()